
# **Limpeza dos Dados**

Começando a limpeza dos dados, vamos agora:

* Remover duplicatas;

* Remover valores missing;

* Descartar variáveis com apenas um ou com quantidades muito altas de valores únicos, como mencionado anteriormente¹;

* Preencher os valores nulos na coluna `CONFERIR` com `False`, considerando assim que são pessoas sem perfil nessa rede social. Ainda, vamos substituir os valores `True` e `False` nessa coluna por `Sim` e `Não`;

* Substituir valores `CONFERIR` e `CONFERIR` na coluna `CONFERIR` por `CONFERIR` e `CONFERIR`, respectivamente.

* Descartar linha onde a variável alvo está ausente e transforma os valores da variável alvo para 0 e 1.

* Gerar dados sintéticos com SMOTE para equilibrar a variável target²


¹É o melhor resultado conhecido

²Aparentemente piora o modelo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import confusion_matrix

In [ ]:
# Conferindo valores null
base.isnull().sum()

# Percentual de null por coluna
(COLUNA.isnull().sum() / COLUNA.shape[0] * 100).sort_values(ascending=False)

# Removendo valores null
base_clean = base.COLUNA.fillna(0) # não tem
df.dropna(inplace=True) # não tem

# Remover duplicatas
base_clean = base_clean.drop_duplicates()

# Descartando variáveis
drop = ['VARIAVEL1', 'VARIAVEL2']

base_clean = base.drop(labels=drop, axis=1)

# Lidando com os valores inf
base_clean = base_clean[base_clean['reported_income'] != np.inf]

# Lidando com os valores negativos
base_clean.loc[base_clean['VARIAVEL1'] < 0, 'VARIAVEL2'] = np.nan

# Transformando a coluna para Sim ou Não (apenas para visualização) 
base_clean['COLUNA'].fillna(value=False, inplace=True, axis=0)
base_clean['COLUNA'] = base_clean['COLUNA'].map({True: 'Sim', False: 'Não'})

# Substituindo valores na coluna email 
base_clean.loc[base_clean['COLUNA'] == 'CONTEUDO', 'COLUNA'] = 'NOVOCONTEUDO'
base_clean.loc[base_clean['COLUNA'] == 'CONTEUDO', 'COLUNA'] = 'NOVOCONTEUDO'

# Limpando a variável alvo
credito_clean.dropna(subset=['target_default'], inplace=True)
credito_clean['target_default'] = credito_clean['target_default'].map({True: 1, False: 0})

credito_clean.head()



# **Tratamento dos Dados**

Os valores ausentes nas colunas numéricas serão preenchidos com a mediana da coluna. Já as colunas categóricas terão os valores ausentes substituídos pelos valores presentes na mesma proporção que estes aparacem na coluna. Para tanto, vamos escrever uma função que:

Recebe uma coluna;

Gera um dicionário onde as chaves são as entradas únicas da coluna e os valores são os percentuais que cada chave representa.

A partir do dicionário, cria uma lista de entradas únicas e uma lista de percentuais;

Usando estas listas, cria uma série do tamanho da quantidade de valore ausentes na coluna contando os valores presentes na proporção correta

Usa a série para preencher os valores ausentes

Verifica se há valores ausentes restantes (por questões de arredondamento dos percentuais) e, se sim, o preenche com o valor mais comum.

Retorna a coluna preenchida.

# **Balanceando os Dados**
# Dividindo e padronizando o dataset original
X = credito_clean.drop('target_default', axis=1)
y = credito_clean['target_default']

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train)
X_train_unb = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Criando o dataset balanceado
maioria = base_clean[credito_clean['target_default'] == 0]
minoria = base_clean[credito_clean['target_default'] == 1]

minoria_balanceada = resample(minoria, replace=True, n_samples=NUMERO)

credito_balanceado = pd.concat([maioria, minoria_balanceada])

# Dividindo e padronizando o dataset balanceado
X_balanceado = credito_balanceado.drop('target_default', axis=1)
y_balanceado = credito_balanceado['target_default']

X_train_balanceado, X_test_balanceado, y_train_balanceado, y_test_balanceado = train_test_split(X_balanceado, y_balanceado)

scaler_balanceado = StandardScaler()
scaler_balanceado.fit(X_train_balanceado)
X_train_balanceado = scaler_balanceado.transform(X_train_balanceado)
X_test_balanceado = scaler_balanceado.transform(X_test_balanceado)

# Verificando o balanceamento
print(credito_balanceado['target_default'].value_counts(), '\n')

fig, ax = plt.subplots(figsize=(8,6))
sns.countplot(base_balanceada['target_default'])
ax.set_title('Dados após o balanceamento')
plt.show()

In [ ]:
def preencher_proporcional(col):
    """ Preenche valores ausentes na mesma proporção dos valores presentes

    Recebe uma coluna e retorna a coluna com os valores ausentes preenchidos
    na proporção dos valores previamente existentes."""
    
    # Gerando o dicionário com valores únicos e sua porcentagens
    percentages = col.value_counts(normalize=True).to_dict()

    # Tranformando as chaves e valores do dicionário em listas      
    percent = [percentages[key] for key in percentages]
    labels = [key for key in percentages]

    # Utilizando as listas para prencher os valores nulos na proporção correta 
    s = pd.Series(np.random.choice(labels, p=percent, size=col.isnull().sum()))
    col = col.fillna(s)
    
    # Verificando se todos os valores ausentes foram preenchidos e
    # preenchendo os que não tiverem sido
    if len(col.isnull()) > 0:
        col.fillna(value=max(percentages, key=percentages.get), inplace=True, axis=0)
        
    return col

## **Modelo**

Teste com quatro algoritmos diferentes para treinar modelos:
* [Regressão Logística](https://pt.wikipedia.org/wiki/Regress%C3%A3o_log%C3%ADstica);
* [Árvores de Decisão](https://medium.com/machine-learning-beyond-deep-learning/%C3%A1rvores-de-decis%C3%A3o-3f52f6420b69);
* [Random Forest](https://en.wikipedia.org/wiki/Random_forest);
* [XGBoost](https://pt.wikipedia.org/wiki/Xgboost).

Usando a técnica de [validação cruzada](https://pt.wikipedia.org/wiki/Valida%C3%A7%C3%A3o_cruzada#:~:text=A%20valida%C3%A7%C3%A3o%20cruzada%20%C3%A9%20uma,da%20modelagem%20%C3%A9%20a%20predi%C3%A7%C3%A3o.), vamos ver o desempenho desses quatro algoritmos tanto com os dados balanceados, como com os dados desbalanceados e, portando, teremos oito modelos.
Usaremos as seguintes [métricas](https://www.mariofilho.com/as-metricas-mais-populares-para-avaliar-modelos-de-machine-learning/) para avaliar os modelos:

* Acurácia;
* Precisão;
* Recall;
* Área sob a curva ROC.

Em seguida, vamos gerar um DataFrame com os resultados de cada um dos modelos para facilitar a visualização e a escolha do melhor modelo.

# **Iterando no DataFrame**

Preencher as variáveis categóricas utilizando a função acima;

Preencher variáveis numéricas com a mediana.

In [ ]:
for col in base_clean.iloc[:,1:].columns.tolist():
    if base_clean[col].dtypes == 'O':
        base_clean[col] = preencher_proporcional( base_clean[col])
    else:
        base_clean[col].fillna(value= base_clean[col].median(), inplace=True, axis=0)

In [ ]:
# Preenchendo com a média, ao invés da mediana, para testar

for col in base_clean.iloc[:,1:].columns.tolist():
    if base_clean[col].dtypes == 'O':
        base_clean[col] = preencher_proporcional( base_clean[col])
    else:
        base_clean[col].fillna(value= base_clean[col].mean(), inplace=True, axis=0)

In [ ]:
# Sem valores null

base_clean.isnull().sum()

# **Modelo de Validação Cruzada**

In [ ]:
# Criando os modelos utilizando validação cruzada
# Não precisamos, já que o modelo do Dani tá ok
logreg_balanceado  = cross_validate(LogisticRegression(solver='liblinear'), X_train_balanceado, y_train_balanceado, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])
logreg = cross_validate(LogisticRegression(solver='liblinear'), X_train, y_train, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])

trees_balanceado  = cross_validate(DecisionTreeClassifier(), X_train_balanceado, y_train_balanceado, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])
trees = cross_validate(DecisionTreeClassifier(), X_train, y_train, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])

forest_balanceado  = cross_validate(RandomForestClassifier(), X_train_balanceado, y_train_balanceado, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])
forest = cross_validate(RandomForestClassifier(), X_train, y_train, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])

xgb_balanceado  = cross_validate(XGBClassifier(), X_train_balanceado, y_train_balanceado, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])
xgb = cross_validate(XGBClassifier(), X_train, y_train, cv=5, scoring=['accuracy', 'precision', 'recall', 'roc_auc'])

# Gerando um DataFrame com os resultados de cada modelo
summary = pd.DataFrame({
            'labels': ['accuracy', 'precision', 'recall', 'roc_auc'],
            'logreg_balanceado': [logreg_balanceado['test_accuracy'].mean(), logreg_balanceado['test_precision'].mean(), logreg_balanceado['test_recall'].mean(), logreg_balanceado['test_roc_auc'].mean()],
            'logreg': [logreg['test_accuracy'].mean(), logreg['test_precision'].mean(), logreg['test_recall'].mean(), logreg['test_roc_auc'].mean()],
            'trees_balanceado': [trees_balanceado['test_accuracy'].mean(), trees_balanceado['test_precision'].mean(), trees_balanceado['test_recall'].mean(), trees_balanceado['test_roc_auc'].mean()],
            'trees': [trees['test_accuracy'].mean(), trees['test_precision'].mean(), trees['test_recall'].mean(), trees['test_roc_auc'].mean()],
            'forest_balanceado': [forest_balanceado['test_accuracy'].mean(), forest_balanceado['test_precision'].mean(), forest_balanceado['test_recall'].mean(), forest_balanceado['test_roc_auc'].mean()],
            'forest': [forest['test_accuracy'].mean(), forest['test_precision'].mean(), forest['test_recall'].mean(), forest['test_roc_auc'].mean()],
            'xgb_balanceado': [xgb_balanceado['test_accuracy'].mean(), xgb_balanceado['test_precision'].mean(), xgb_balanceado['test_recall'].mean(), xgb_balanceado['test_roc_auc'].mean()],
            'xgb': [xgb['test_accuracy'].mean(), xgb['test_precision'].mean(), xgb['test_recall'].mean(), xgb['test_roc_auc'].mean()]           
}).set_index('labels')
summary.index.name=None
summary = summary.transpose()    
summary.style.applymap(lambda x: 'background-color: lightgreen' if x >= 0.75 else '')